# Using Zephyr 7B Beta Quantised Model

* [TheBloke/zephyr-7B-beta-GGUF](https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF)
* Used CTransformers wrapper

In [ ]:
%pip install torch==2.2.1
%pip install langchain==0.1.9
%pip install langchain-community==0.0.24
%pip install ctransformers==0.2.27
%pip install streamlit==1.31.1
%pip install streamlit-extras==0.4.0
%pip install langchain==0.1.9
%pip install rank_bm25==0.2.2
%pip install pypdf==4.0.2
%pip install chromadb==0.4.24
%pip install tiktoken==0.6.0

In [1]:
import os
from langchain_community.llms import CTransformers
from langchain import PromptTemplate, LLMChain

In [2]:
model_type = "mistral"
model_id = "TheBloke/zephyr-7B-beta-GGUF"
model_file = "zephyr-7b-beta.Q4_K_S.gguf"

In [3]:
config = {
    "max_new_tokens": 1024,
    "repetition_penalty": 1.1,
    "temperature": 1,
    "top_k": 50,
    "top_p": 0.9,
    "stream": True,
    "threads": int(os.cpu_count() / 2),
}

In [4]:
init_model = CTransformers(model=model_id, model_file=model_file, model_type=model_type, **config, lib="avx2")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

zephyr-7b-beta.Q4_K_S.gguf:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/fe/17/fe17596731f84a0d03bece77489780bc7e068323c0aeca88b6393d3e9e65dd49/cafa0b85b2efc15ca33023f3b87f8d0c44ddcace16b3fb608280e0eb8f425cb1?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27zephyr-7b-beta.Q4_K_S.gguf%3B+filename%3D%22zephyr-7b-beta.Q4_K_S.gguf%22%3B&Expires=1709696299&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTY5NjI5OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ZlLzE3L2ZlMTc1OTY3MzFmODRhMGQwM2JlY2U3NzQ4OTc4MGJjN2UwNjgzMjNjMGFlY2E4OGI2MzkzZDNlOWU2NWRkNDkvY2FmYTBiODViMmVmYzE1Y2EzMzAyM2YzYjg3ZjhkMGM0NGRkY2FjZTE2YjNmYjYwODI4MGUwZWI4ZjQyNWNiMT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=JJymveuF19P%7EYsnDVvRKvHbsUjRrNso4dCIEhQ591C6Ponli%7EvQZXE3jKIWNH0ZG%7El1ERzgSns5Qdhx9ImLRLyCtq0szMjeb7eycm%7E8BBpBH3%7EUle4RQoGm1056cJbbOqbiCyTQpFsoRe6N3ivAxTn11BjMY1b-dAmZnWbL%7E%7EyyY3Og7h9YVXX3g%7E-3I5FaWIwv-GTwPP

zephyr-7b-beta.Q4_K_S.gguf:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/fe/17/fe17596731f84a0d03bece77489780bc7e068323c0aeca88b6393d3e9e65dd49/cafa0b85b2efc15ca33023f3b87f8d0c44ddcace16b3fb608280e0eb8f425cb1?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27zephyr-7b-beta.Q4_K_S.gguf%3B+filename%3D%22zephyr-7b-beta.Q4_K_S.gguf%22%3B&Expires=1709696299&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTY5NjI5OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ZlLzE3L2ZlMTc1OTY3MzFmODRhMGQwM2JlY2U3NzQ4OTc4MGJjN2UwNjgzMjNjMGFlY2E4OGI2MzkzZDNlOWU2NWRkNDkvY2FmYTBiODViMmVmYzE1Y2EzMzAyM2YzYjg3ZjhkMGM0NGRkY2FjZTE2YjNmYjYwODI4MGUwZWI4ZjQyNWNiMT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=JJymveuF19P%7EYsnDVvRKvHbsUjRrNso4dCIEhQ591C6Ponli%7EvQZXE3jKIWNH0ZG%7El1ERzgSns5Qdhx9ImLRLyCtq0szMjeb7eycm%7E8BBpBH3%7EUle4RQoGm1056cJbbOqbiCyTQpFsoRe6N3ivAxTn11BjMY1b-dAmZnWbL%7E%7EyyY3Og7h9YVXX3g%7E-3I5FaWIwv-GTwPP

zephyr-7b-beta.Q4_K_S.gguf:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

## Without Prompt Template

In [5]:
query = "what is the meaning of the life ?"

In [6]:
result = init_model(query)
print(result)

/mnt/d/repo/experiments/chat_with_pdf_using_zephyr-7b/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




what happens after we die ?

is there any god or creator ?

who am I really ?

these are the questions that have always fascinated human mind and kept us thinking for ages. These questions are so profound, yet simple and so personal. We all have our own answers to these questions, whether in form of religion, spirituality or philosophy, which become a part of our life philosophy as we grow up.

But there is another dimension where people look beyond the boundaries of these religions and philosophies. They go into a quest for truth that goes deeper than what they have been taught by their religion or philosophy. They start looking within themselves to find the answers. This quest takes them on a journey of self-discovery, which is often referred to as Spirituality.

Spirituality, at its core, is an intense thirst to know the truth about life and ourselves. It is a longing for connection with something greater than oneself – God or the Universe. The spiritual quest takes us on a journe

## With Prompt Template

In [7]:
template = """You are a helpful AI Assistant that follows instructions extremely well.
Question: {question}

Answer: Let's think step by step and answer it faithfully.
"""

In [8]:
prompt = PromptTemplate(template=template, input_variables=["question"])

In [9]:
chain = LLMChain(prompt=prompt, llm=init_model, verbose=True)

In [10]:
query = "What is LLM ?"

In [11]:
result = chain.run(query)

/mnt/d/repo/experiments/chat_with_pdf_using_zephyr-7b/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful AI Assistant that follows instructions extremely well.
Question: What is LLM ?

Answer: Let's think step by step and answer it faithfully.


> Finished chain.


In [13]:
print(result)


LLM stands for Large Language Model. It refers to a type of machine learning algorithm specifically designed to process and generate human-like language, typically in the form of text or speech. These models are called "large" because they require vast amounts of training data to learn the complex patterns and relationships within language. The ultimate goal of LLMs is to enable more natural and intuitive interactions between humans and machines through enhanced communication capabilities.


## RAG - Talk to PDF

In [14]:
import os
from langchain_community.llms import CTransformers
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.retrievers import BM25Retriever, EnsembleRetriever

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

### Load Data

In [21]:
file_path = "../data/Orca Progressive Learning from Complex.pdf"
data_file = PyPDFLoader(file_path)
docs = data_file.load()

### Split & Chunk Docs

In [22]:
# create chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_documents(docs)

### Load Embedder

In [23]:
HF_TOKEN = input("Enter your HuggingFace Token")

In [24]:
# https://huggingface.co/BAAI/bge-base-en-v1.5
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_TOKEN, model_name="BAAI/bge-base-en-v1.5"
)

### Retrievers

In [25]:
# retrieve k
k = 5

#### Vector Retriever

In [26]:
vector_store = Chroma.from_documents(chunks, embeddings)
vector_retriever = vector_store.as_retriever(search_kwargs={"k": k})

#### Semantic Retriever

In [27]:
semantic_retriever = BM25Retriever.from_documents(chunks)
semantic_retriever.k = k

#### Ensemble Retriever

In [28]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, semantic_retriever], weights=[0.5, 0.5]
)

### Init LLM Model

In [29]:
model_type = "mistral"
model_id = "TheBloke/zephyr-7B-beta-GGUF"
model_file = "zephyr-7b-beta.Q4_K_S.gguf"

In [49]:
config = {
    "max_new_tokens": 2048,
    "repetition_penalty": 1.1,
    "temperature": 1,
    "top_k": 50,
    "top_p": 0.9,
    "stream": True,
    "context_length": 4096,
    "gpu_layers": 0,
    "threads": int(os.cpu_count() / 2),
}

In [50]:
llm = CTransformers(
    model=model_id, model_file=model_file, model_type=model_type, config=config, lib="avx2"
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

### Prompting

In [51]:
template = """You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. 
You will get a $100 tip if you provide correct answer.

Context: {context}

Question: {question}

Answer: Let's think step by step and answer it faithfully.
"""

In [52]:
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

In [53]:
chain = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [54]:
print(chain.invoke("What is instruction tuning?"))


Instruction tuning is a technique that allows pre-trained language models to learn from input (natural language descriptions of the task) and response pairs, for example, "{\"instruction\": \"Arrange the words in the given sentence to form a grammatically\ncorrect sentence.\", \"input\": \"the quickly brown fox jumped\", \"output\": \"the brown\nfox jumped quickly\"} .". It is commonly used for both language-only and multimodal tasks, such as image captioning and visual question answering. In recent times, many works have adopted instruction tuning to train smaller language models with outputs generated from large foundation models like GPT family. However, these approaches face several challenges, including limited task diversity, query complexity, and small-scale training data that understate the benefits of such methods. The Orca model presented in this thesis addresses these limitations by combining self-supervised learning, reinforcement learning, and instruction tuning to achiev